Imports

In [1]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler, AutoModelForCausalLM
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
import evaluate


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /home/arjun/NewPytorchEnv/bin/python3.10: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizer

/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('0'), PosixPath('1')}
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('@/tmp/.ICE-unix/1592,unix/PC'), PosixPath('local/PC')}
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path wer

Dataset preparation and tokenising

In [2]:
dataset = load_dataset("tatsu-lab/alpaca")

Using custom data configuration tatsu-lab--alpaca-2b32f0433506ef5f
Found cached dataset parquet (/home/arjun/.cache/huggingface/datasets/tatsu-lab___parquet/tatsu-lab--alpaca-2b32f0433506ef5f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset['train'].features

{'instruction': Value(dtype='string', id=None),
 'input': Value(dtype='string', id=None),
 'output': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [4]:
dataset = dataset['train']
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 52002
})

In [5]:
# Making dataset smaller for fast training
dataset = dataset.select(range(1000))

In [6]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

new_dataset = {'labels': [], 'output': []}

for example in dataset:
    input_text = example['text'].replace('\n', '')
    input_list = input_text.split('###')
    input_ids = tokenizer(input_list[0] + input_list[1], truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    # output_id s = tokenizer(input_list[2], truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    new_dataset['labels'].append(input_ids['input_ids'])
    # new_dataset['output'].append(output_ids['input_ids'])

new_dataset = Dataset.from_dict(new_dataset)
new_dataset.set_format("torch")

ArrowInvalid: Column 1 named output expected length 1000 but got length 0

In [ ]:
new_dataset.features

DataLoader

In [ ]:
dataloader = DataLoader(new_dataset,shuffle=True, batch_size=8)

Loading model

In [ ]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

Optimiser and scheduler

In [ ]:
num_epochs = 10

optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model.to(device)


In [ ]:
for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids = batch['labels'].to(device)  # Get input_ids from batch
        outputs = model(input_ids=input_ids)   # Use only input_ids for text generation
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        # progress_bar.update(1)

In [ ]:
progress_bar = tqdm(range(num_training_steps),desc='Training', unit='steps')

model.train()   # Some layers behave differently to training and inference. This sets all those 
                 # layers into training mode
for epoch in range(num_epochs):
    for batch in dataloader:
        print(batch)
        batch = {x: y.to(device) for x, y in batch.items()} # Put tokenised text to GPU
        outputs = model(**batch) # **batch allows you to unpack the key-value pairs from 
                                  # dictionary batch into the correct parameter space of model()
        loss = outputs.loss      # compute loss 
        loss.backward()          # computes gradients
        optimizer.step()         # optimises
        lr_scheduler.step()      # updates lr according to schedule. Improves performance
        optimizer.zero_grad()    # resets the gradients
        progress_bar.update(1)   # updates progress bar by 1